In [1]:
import aoareader as reader
import torch
import time

We here create a class to have all paraeters.It is for the case that we want to train from scratch.
For convinience, we set the valid data as the test data.
To fully understand the parameters, please refer to file train.py


In [2]:
class opt(object):
    def __init__(self):
        self.traindata='data/cnn/train.txt.pt'
        self.validdata='data/cnn/test.txt.pt'
        self.dict='data/cnn/dict.pt'
        self.save_model='model_cnn'
        self.train_from=''
        self.hidden_size=384
        self.embed_size=384
        self.batch_size=64
        self.dropout=0.1
        self.start_epoch=1
        self.epochs=13
        self.learning_rate=0.001
        self.weight_decay=0.0001
        self.gpu=0
        self.log_interval=100
opt=opt()
print(vars(opt))
if opt.gpu:
    torch.cuda.set_device(opt.gpu)

{'traindata': 'data/cnn/train.txt.pt', 'validdata': 'data/cnn/test.txt.pt', 'dict': 'data/cnn/dict.pt', 'save_model': 'model_cnn', 'train_from': '', 'hidden_size': 384, 'embed_size': 384, 'batch_size': 64, 'dropout': 0.1, 'start_epoch': 1, 'epochs': 13, 'learning_rate': 0.001, 'weight_decay': 0.0001, 'gpu': 0, 'log_interval': 100}


Define the loss function and evalutation function

In [3]:
def loss_func(answers, pred_answers, answer_probs):
    num_correct = (answers == pred_answers).sum().squeeze().data[0]
    loss = - torch.mean(torch.log(answer_probs),0, keepdim=True)
    return loss.cuda(), num_correct

In [4]:
def eval(model, data):
    total_loss = 0.0
    total = 0.0
    total_correct = 0.0

    model.eval()
    for i in range(len(data)):
        (batch_docs, batch_docs_len, doc_mask), (batch_querys, batch_querys_len, query_mask), batch_answers, candidates = data[i]

        pred_answers, probs = model(batch_docs, batch_docs_len, doc_mask,
                                    batch_querys, batch_querys_len, query_mask,
                                    answers=batch_answers, candidates=candidates)

        loss, num_correct = loss_func(batch_answers, pred_answers, probs)

        total_in_minibatch = batch_answers.size(0)
        total_loss += loss.data[0] * total_in_minibatch
        total_correct += num_correct
        total += total_in_minibatch
        del loss, pred_answers, probs

    model.train()
    return total_loss / total, total_correct.type(torch.float) / total

In [5]:
def trainModel(model, trainData, validData, optimizer: torch.optim.Adam):
    print(model)
    start_time = time.time()

    def trainEpoch(epoch):
        trainData.shuffle()

        total_loss, total, total_num_correct = 0, 0, 0
        report_loss, report_total, report_num_correct = 0, 0, 0
        for i in range(len(trainData)):
            (batch_docs, batch_docs_len, doc_mask), (batch_querys, batch_querys_len, query_mask), batch_answers, candidates = trainData[i]

            model.zero_grad()
            pred_answers, answer_probs = model(batch_docs, batch_docs_len, doc_mask, batch_querys, batch_querys_len, query_mask,answers=batch_answers, candidates=candidates)

            loss, num_correct = loss_func(batch_answers, pred_answers, answer_probs)

            loss.backward()
            for parameter in model.parameters():
                parameter.grad.data.clamp_(-5.0, 5.0)
            # update the parameters
            optimizer.step()

            total_in_minibatch = batch_answers.size(0)

            report_loss += loss.data[0] * total_in_minibatch
            report_num_correct += num_correct
            report_total += total_in_minibatch

            total_loss += loss.data[0] * total_in_minibatch
            total_num_correct += num_correct
            total += total_in_minibatch
            if i % opt.log_interval == 0:
                print("Epoch %2d, %5d/%5d; avg loss: %.2f; acc: %2.2f;  %6.0f s elapsed" %
                      (epoch, i+1, len(trainData),
                       report_loss / report_total,
                       (100.0*report_num_correct / report_total),
                       time.time()-start_time))
                valid_loss, valid_acc = eval(model, validData)
                print('=' * 20)
                print('Evaluating on validation set:')
                print('Validation loss: %.2f' % valid_loss)
                print('Validation accuracy: %2.2f' % (valid_acc * 100.0))
                print('=' * 20)

                report_loss = 0.0
                report_total = 0.0
                report_num_correct = 0.0
            del loss, pred_answers, answer_probs

        return total_loss / total, total_num_correct / total

    for epoch in range(1, opt.epochs + 1):
        print('')

        #  (1) train for one epoch on the training set
        train_loss, train_acc = trainEpoch(epoch)
        print('Epoch %d:\t average loss: %.2f\t train accuracy: %g' % (epoch, train_loss, train_acc*100))

        #  (2) evaluate on the validation set
        valid_loss, valid_acc = eval(model, validData)
        print('=' * 20)
        print('Evaluating on validation set:')
        print('Validation loss: %.2f' % valid_loss)
        print('Validation accuracy: %2.2f' % (valid_acc*100.0))
        print('=' * 20)

        model_state_dict = model.state_dict()
        optimizer_state_dict = optimizer.state_dict()
        #  (4) drop a checkpoint
        checkpoint = {
            'model': model_state_dict,
            'epoch': epoch,
            'optimizer': optimizer_state_dict,
            'opt': opt,
        }
        torch.save(checkpoint,
                   'model_cnn/%s_epoch%d_acc_%.2f.pt' % (opt.save_model, epoch, 100*valid_acc))

Here we set our model train from a pretrained model

In [6]:
opt.train_from='model_cnn/model_cnn_epoch17_acc_69.48.pt'
train_from = opt.train_from
if opt.train_from:
    train_from = True
    checkpoint = torch.load(opt.train_from)
    opt = checkpoint['opt']
    opt.hidden_size=opt.gru_size

print("Loading dictrionary from ", opt.dict)
vocab_dict = torch.load(opt.dict)
print("Loading train data from ", opt.traindata)
train_data = torch.load(opt.traindata)
print("Loading valid data from ", opt.validdata)
valid_data = torch.load(opt.validdata)

train_dataset = reader.Dataset(train_data, opt.batch_size, True)
valid_dataset = reader.Dataset(valid_data, opt.batch_size, True, volatile=True)

print(' * vocabulary size = %d' %
      (vocab_dict.size()))
print(' * number of training samples. %d' %
      len(train_data['answers']))
print(' * maximum batch size. %d' % opt.batch_size)

print('Building model...')

model = reader.AoAReader(vocab_dict, dropout_rate=opt.dropout, embed_dim=opt.embed_size, hidden_dim=opt.hidden_size)
# no way on CPU
model.cuda()
# for parameter in model.parameters():
#     print(parameter)
if train_from:
    print('Loading model from checkpoint at %s' % opt.train_from)
    chk_model = checkpoint['model']
    model.load_state_dict(chk_model)
    opt.start_epoch = checkpoint['epoch'] + 1

optimizer = torch.optim.Adam(model.parameters(), lr=opt.learning_rate, weight_decay=opt.weight_decay)

if train_from:
    optimizer.load_state_dict(checkpoint['optimizer'])

nParams = sum([p.nelement() for p in model.parameters()])
print('* number of parameters: %d' % nParams)

trainModel(model, train_dataset, valid_dataset, optimizer)

Loading dictrionary from  data/cnn/dict.pt
Loading train data from  data/cnn/train.txt.pt
Loading valid data from  data/cnn/test.txt.pt
 * vocabulary size = 119656
 * number of training samples. 380298
 * maximum batch size. 64
Building model...


/home/ttnguyen/Projects/Transformer_Encoder_AoAreader/aoareader/AoAReader.py:54: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
  weigth_init.orthogonal(weight.data)


Loading model from checkpoint at 
* number of parameters: 46835392
AoAReader(
  (embedding): Embedding(119656, 384, padding_idx=0)
  (transformer): Encoder(
    (embedding_proj): Linear(in_features=384, out_features=384, bias=False)
    (enc): Sequential(
      (0): EncoderLayer(
        (multi_head_attention): MultiHeadAttention(
          (query_linear): Linear(in_features=384, out_features=384, bias=False)
          (key_linear): Linear(in_features=384, out_features=384, bias=False)
          (value_linear): Linear(in_features=384, out_features=384, bias=False)
          (output_linear): Linear(in_features=384, out_features=384, bias=False)
          (dropout): Dropout(p=0.0)
        )
        (positionwise_feed_forward): PositionwiseFeedForward(
          (layers): ModuleList(
            (0): Conv(
              (pad): ConstantPad1d(padding=(1, 1), value=0)
              (conv): Conv1d(384, 64, kernel_size=(3,), stride=(1,))
            )
            (1): Conv(
              (pad)

/home/ttnguyen/anaconda3/envs/py36torch/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  


Epoch  1,     1/ 5943; avg loss: 1.05; acc: 65.00;       2 s elapsed


...

Evaluating on validation set:
Validation loss: 1.02
Validation accuracy: 69.48
Epoch  2,  2701/ 5943; avg loss: 1.04; acc: 70.00;    8058 s elapsed
Evaluating on validation set:
Validation loss: 1.02
Validation accuracy: 69.01
Epoch  2,  2801/ 5943; avg loss: 1.03; acc: 70.00;    8153 s elapsed
Evaluating on validation set:
Validation loss: 1.02
Validation accuracy: 69.67
Epoch  2,  2901/ 5943; avg loss: 1.06; acc: 69.00;    8248 s elapsed
Evaluating on validation set:
Validation loss: 1.02
Validation accuracy: 69.73
Epoch  2,  3001/ 5943; avg loss: 1.04; acc: 68.00;    8341 s elapsed
Evaluating on validation set:
Validation loss: 1.02
Validation accuracy: 69.54
Epoch  2,  3101/ 5943; avg loss: 1.04; acc: 69.00;    8432 s elapsed
Evaluating on validation set:
Validation loss: 1.01
Validation accuracy: 70.08
Epoch  2,  3201/ 5943; avg loss: 1.04; acc: 69.00;    8525 s elapsed
Evaluating on validation set:
Validation loss: 1.01
Validation accuracy: 69.98
Epoch  2,  3301/ 5943; avg loss: 

Epoch  3,   901/ 5943; avg loss: 0.96; acc: 72.00;   11946 s elapsed
Evaluating on validation set:
Validation loss: 1.04
Validation accuracy: 69.45
Epoch  3,  1001/ 5943; avg loss: 1.01; acc: 70.00;   12044 s elapsed
Evaluating on validation set:
Validation loss: 1.03
Validation accuracy: 69.76
Epoch  3,  1101/ 5943; avg loss: 0.96; acc: 71.00;   12135 s elapsed
Evaluating on validation set:
Validation loss: 1.03
Validation accuracy: 70.04
Epoch  3,  1201/ 5943; avg loss: 0.95; acc: 71.00;   12227 s elapsed
Evaluating on validation set:
Validation loss: 1.04
Validation accuracy: 69.64
Epoch  3,  1301/ 5943; avg loss: 0.99; acc: 70.00;   12319 s elapsed
Evaluating on validation set:
Validation loss: 1.03
Validation accuracy: 69.73
Epoch  3,  1401/ 5943; avg loss: 0.97; acc: 70.00;   12412 s elapsed
Evaluating on validation set:
Validation loss: 1.02
Validation accuracy: 70.26
Epoch  3,  1501/ 5943; avg loss: 1.01; acc: 70.00;   12503 s elapsed
Evaluating on validation set:
Validation lo

Evaluating on validation set:
Validation loss: 1.02
Validation accuracy: 69.86
Epoch  3,  5301/ 5943; avg loss: 1.02; acc: 70.00;   16021 s elapsed
Evaluating on validation set:
Validation loss: 1.02
Validation accuracy: 69.76
Epoch  3,  5401/ 5943; avg loss: 1.02; acc: 69.00;   16112 s elapsed
Evaluating on validation set:
Validation loss: 1.01
Validation accuracy: 70.17
Epoch  3,  5501/ 5943; avg loss: 1.02; acc: 69.00;   16206 s elapsed
Evaluating on validation set:
Validation loss: 1.02
Validation accuracy: 69.89
Epoch  3,  5601/ 5943; avg loss: 1.03; acc: 70.00;   16300 s elapsed
Evaluating on validation set:
Validation loss: 1.02
Validation accuracy: 69.48
Epoch  3,  5701/ 5943; avg loss: 1.04; acc: 69.00;   16392 s elapsed
Evaluating on validation set:
Validation loss: 1.02
Validation accuracy: 70.01
Epoch  3,  5801/ 5943; avg loss: 1.04; acc: 69.00;   16485 s elapsed
Evaluating on validation set:
Validation loss: 1.02
Validation accuracy: 69.14
Epoch  3,  5901/ 5943; avg loss: 

Epoch  4,  3501/ 5943; avg loss: 1.01; acc: 70.00;   19905 s elapsed
Evaluating on validation set:
Validation loss: 1.02
Validation accuracy: 70.01
Epoch  4,  3601/ 5943; avg loss: 1.00; acc: 70.00;   20001 s elapsed
Evaluating on validation set:
Validation loss: 1.02
Validation accuracy: 69.98
Epoch  4,  3701/ 5943; avg loss: 0.99; acc: 70.00;   20092 s elapsed
Evaluating on validation set:
Validation loss: 1.02
Validation accuracy: 69.92
Epoch  4,  3801/ 5943; avg loss: 0.99; acc: 70.00;   20182 s elapsed
Evaluating on validation set:
Validation loss: 1.01
Validation accuracy: 70.26
Epoch  4,  3901/ 5943; avg loss: 1.02; acc: 70.00;   20276 s elapsed
Evaluating on validation set:
Validation loss: 1.01
Validation accuracy: 69.57
Epoch  4,  4001/ 5943; avg loss: 1.05; acc: 68.00;   20370 s elapsed
Evaluating on validation set:
Validation loss: 1.02
Validation accuracy: 69.54
Epoch  4,  4101/ 5943; avg loss: 1.00; acc: 70.00;   20461 s elapsed
Evaluating on validation set:
Validation lo

KeyboardInterrupt: 

So in the end, the accuracy of the Transformer Encoder Attention over Attention is 70.86%.
It is comparable with the bidirectional gru which has the accuracy of 69.89%.
For more details, please refer to the github link @placeholder; see the file nohup.out
However with the advantage of transformer model, you can see that the speed is faster:
One epoch in the former took 5582s which is 1 and half hour while in the later took 13874s which is 3 hours and half with the similar accuracy.
One more thing to note is that they have the similar number of parameters: the former has 46835392 parameters and the later has 47721984 parameters